In [4]:
from ctrace import *
import pandas as pd
import numpy as np

In [61]:
# Load in results file
with open(PROJECT_ROOT /"output"/"run_Hmmfn/" / "results.csv", "r") as csv:
    df = pd.read_csv(csv)
print(len(df))
df.head()

12000


,Unnamed: 0,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap
0,0,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,94.015330,139.4,0.569600,661.962407,1
1,1,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,1217,6233,11135,16931,28,93.797013,118.6,0.574026,661.962407,5
2,2,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,85.069098,104.8,0.032785,661.962407,-1
3,3,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,1217,6233,11135,16931,28,85.069098,120.8,0.032597,661.962407,-1
4,4,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,1217,6233,11135,16931,28,84.819447,125.4,0.808267,661.962407,64


In [62]:
# Extract the Gurobi MILP optimal runs
optimal = df[df["method"] == "mip_gurobi"]
print(len(optimal))
optimal.head()

3000


,Unnamed: 0,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap
6,6,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,84.156001,113.2,0.832709,661.962407,72
11,11,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,1217,6233,11135,16931,28,84.156001,108.8,0.862365,661.962407,72
20,20,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,2359,25664,32967,145552,59,497.959798,1340.0,13.710341,3733.643370,56
22,22,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,2359,25664,32967,145552,59,497.959798,1393.6,14.214940,3733.643370,56
27,27,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,4854,31285,22040,82500,32,407.842102,945.4,8.959507,5702.689558,45


In [63]:
# Remove duplicates from different trials
optimal = optimal[optimal["trial_id"] == 0]

# Only these 4 ("G", "p", "budget", "from_cache") are needed to uniquely identify simulation parameters
optimal = optimal[["G", "p", "budget", "from_cache", "mip_value"]]
optimal = optimal.rename(columns={"mip_value": "optimal_mip_value"})

print(len(optimal))
optimal.head()

1500


,G,p,budget,from_cache,optimal_mip_value
6,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,84.156001
20,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,497.959798
28,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,407.842102
30,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,280.462819
48,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,300.202036


In [64]:
# Merge on the aforementioned key columns
df = df.merge(optimal, on=["G", "p", "budget", "from_cache"])

In [65]:
df.head()


,Unnamed: 0,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value
0,0,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,94.015330,139.4,0.569600,661.962407,1,84.156001
1,1,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,1217,6233,11135,16931,28,93.797013,118.6,0.574026,661.962407,5,84.156001
2,2,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,85.069098,104.8,0.032785,661.962407,-1,84.156001
3,3,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,1217,6233,11135,16931,28,85.069098,120.8,0.032597,661.962407,-1,84.156001
4,4,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,1217,6233,11135,16931,28,84.819447,125.4,0.808267,661.962407,64,84.156001


In [66]:
df["mip_ratio_gap"] = df["mip_value"] / df["optimal_mip_value"]
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap
0,0,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,94.015330,139.4,0.569600,661.962407,1,84.156001,1.117155
1,1,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,1217,6233,11135,16931,28,93.797013,118.6,0.574026,661.962407,5,84.156001,1.114561
2,2,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,85.069098,104.8,0.032785,661.962407,-1,84.156001,1.010850
3,3,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,1217,6233,11135,16931,28,85.069098,120.8,0.032597,661.962407,-1,84.156001,1.010850
4,4,montgomery,0.078,100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,1217,6233,11135,16931,28,84.819447,125.4,0.808267,661.962407,64,84.156001,1.007884


In [69]:
# Check that mip_ratio_gap >= 1 for all elements
assert len(df[df["mip_ratio_gap"] < 1]) == 0

In [74]:
greedy = df[df["method"] == "greedy_weighted"]
greedy.min()

Unnamed: 0                                                           2
G                                                           montgomery
p                                                                0.078
budget                                                             100
from_cache           /home/ubuntu/ContactTracing/data/SIR_Cache/tim...
method                                                 greedy_weighted
trial_id                                                             0
I_size                                                             527
v1_size                                                           2808
v2_size                                                           6472
num_cross_edges                                                   8273
maxD                                                                19
mip_value                                                      22.6286
min_exposed_value                                                 23.4
durati